***

<center><u><H1>Artificial Neural Network: regressione</H1></u></center>

***

***
# Il dataset

Per la prima ANN che andremo a creare, lavoriamo sulla regressione. Per avere un confronto, riprendiamo il dataset è fuel_consumption.csv, fornito su Virtual Learning Environment e disponibile anche a [questo link](https://www.kaggle.com/sarita19/fuel-consumption), che abbiamo usato qualche lezione fa per costruire dei regressori con ML.

Abbiamo già visto che il dataset non richiede alcuna elaborazione, se ci concentriamo su alcune feature. Questa volta selezioniamo tutte le feature numeriche. Poi andremo a valutare prima il caso con 3 feature e poi quello con tutte.

Importiamo i primi moduli necessari:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("00_datasets/fuel_consumption.csv")
df

Andando a stampare i data type con il metodo info(), vediamo quali sono le feature che vogliamo mantenere:

In [ ]:
df.info()

Per selezionare solo quelle numeriche, possiamo ricorrere al metodo [select_feature()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html):

In [ ]:
columns = df.select_dtypes(include=['int64', 'float64']).columns
df = df[columns]
df

Per quanto riguarda il pre-processing, lo abbiamo già visto nella lezione relativa alle tecniche di regressione con ML, per cui possiamo concentrarci sul design della rete neurale.

# Rete neurale

Come esempio, lavoriamo con 3 sole IV come fatto con la regressione lineare multipla alcune lezioni fa:

In [ ]:
df_red = df[['engine_size','cylinders','fuel_consumption_comb','co2_emissions']]
df_red

## Train/test split e feature scaling

Separiamo training e test set con un metodo holdout, per il momento. Lavoriamo con una divisione al 80%-20%, come fatto con la regressione lineare multipla.

In [ ]:
from sklearn.model_selection import train_test_split

X = df_red.iloc[:, :-1].to_numpy()
y = df_red.iloc[:, -1].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

Per quanto riguarda il feature scaling, lo applichiamo a tutte le feature in ingresso (se ci fossero state delle dummy variable, lo avremmo applicato anche a loro):

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Salviamo il numero di feature in una variabile per comodità nella creazione della rete neurale:

In [ ]:
n_feature = X_train.shape[1]
n_feature

Importiamo TensorFlow per poter visualizzare la rete neurale tra poco:

In [ ]:
import tensorflow as tf

## Design della rete

Si possono creare diversi design della rete modificando molti parametri.

Iniziamo con il comprendere come variano le prestazioni in funzione del numero di hidden layer (ovvero la profondità della rete neurale) e in funzione del numero di neuroni degli hidden layer. Dopo aver compreso come variano le prestazioni, andremo ad ottimizzare la rete neurale con una grid search in modo più mirato.

Importiamo i moduli **models** e **layers** da Keras, che occorrono per creare qualsiasi rete neurale; inoltre, importiamo anche la funzione r2_score per calcolare questa metrica e confrontarla con quanto ottenuto con le tecniche di regressione lineare e polinomiale.

In [ ]:
from keras import models, layers
from sklearn.metrics import r2_score

### 1 hidden layer

Per il numero di nodi, si potrebbe ricorrere alle seguenti strategie:
- soluzione A: $M$ nodi;
- soluzione A: $(M+L)*2/3$ nodi;
- soluzione B: $(M+L)/2$ nodi;
- soluzione C: $\sqrt{ML}$ nodi.

Tuttavia, con 3 nodi in ingresso (davvero pochi parlando di reti neurali, ma come esempio va bene) otteniamo i seguenti valori:

In [ ]:
print(f"Soluzione A: {n_feature}")
print(f"Soluzione B: {int(2*(n_feature+1)/3)}")
print(f"Soluzione C: {int((n_feature+1)/2)}")
print(f"Soluzione D: {int(np.sqrt(n_feature))}")

Un aspetto da tenere a mente è che il valore di nodi passato ad un layer deve essere un intero, quindi si dovrà convertire sempre in int il valore ottenuto con la formula. Dalle stampe riportate sopra, vediamo come la soluzione D sia di scarso interesse (sarebbe come inserire due output layer uno dopo l'altro), mentre la seconda e la terza sono uguali. Valutiamo quindi solo le opzioni A e B.

Per comodità nella valutazione di diverse soluzioni, creiamo una funzione che genera una rete compilata:
- in merito alle funzioni di attivazione, utilizzeremo delle **ReLu** negli hidden layer, soluzione piuttosto tipica nelle reti neurali, almeno come scelta iniziale. Nell'output layer, avendo un problema di regressione, procediamo con una identity function (quindi l'opzione **linear** di Keras);
- avendo un problema di regressione, utilizziamo **mse** come funzione di costo per la backpropagation;
- come ottimizzatore della backpropagation, ricorriamo all'algoritmo **adam**, spesso utilizzato nelle reti neurali;
- inseriamo nel codice anche la stampa del riassunto della nostra rete per comodità, tramite il metodo summary().

In [ ]:
def create_nn_1h(n_neuron: int, summary_flag: bool = True):
    """
    Questa funzione crea una rete neurale con un singolo hidden layer. 
    :param n_neuron: numero di neuroni dell'hidden layer
    :param summary_flag: flag booleano per indicare se mostrare il riassunto della rete oppure no
    :return: modello di rete neurale compilato con ottimizzatore 'adam' e loss function 'mse'
    """
    # layout della rete
    inputs = layers.Input(name = 'input_layer', shape=(n_feature,) )
    h1 = layers.Dense(name = 'hidden_layer_1', units = n_neuron, activation = 'relu')(inputs)
    outputs = layers.Dense(name = 'output_layer', units = 1, activation = 'linear')(h1)
    # creazione della rete
    model = models.Model(inputs = inputs, outputs = outputs, name = 'NN_1HL')
    # summary
    if summary_flag: model.summary(show_trainable=True)
    # compilazione
    model.compile(optimizer = 'adam', loss = 'mse')
    # ritorno 
    return model

#### Soluzione A: $M$ nodi

Creiamo la rete con la funzione appena scritta indicando $M$ nodi nel singolo hidden layer. Poi possiamo visualizzarla e addestrarla con il metodo [fit()](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit).

Prima di procedere, dobbiamo però sottolineare un aspetto chiave per poter interpretare correttamente i risultati. Una rete neurale è inerentemente stocastica, a causa dell'inizializzazione pseudo-casuale dei pesi ogni volta che essa viene compilata (ovvero ogni volta che si chiama il metodo compile su un modello). Questo si aggiunge poi agli eventuali shuffle del dataset e qualsiasi altra forma di randomizzazione, ma quella dei pesi è fondamentale in quanto la stessa identica rete addestrata sullo stesso identico train set potrebbe portare a dei risultati molto diversi perché si può finire in un minimo locale diverso...

Il livello delle oscillazioni nelle prestazioni dipende fortemente dalla qualità dei dati, dalla quantità delle osservazioni e dal design della rete neurale. In linea di principio, si potrebbe ricorrere a diversi approcci per la riduzione della varianza sui modelli allenati, come tecniche di regolarizzazione, layer di dropout, inizializzazioni multiple, tecniche di ensembling, etc. Nel corso non abbiamo modo di analizzare nel dettaglio queste soluzioni, in quanto tecniche avanzate relative alle reti neurali. Per questo motivo, andremo a re-inizializzare il seed tramite la funzione [tf.keras.utils.set_random_seed()](https://www.tensorflow.org/api_docs/python/tf/keras/utils/set_random_seed), che ci permette di resettare **tutti i seed pseudo-casuali**: Python, NumPy, Keras ed eventuali backend come TensorFlow.

In [ ]:
my_seed = 15
tf.keras.utils.set_random_seed(my_seed)

Ora creiamo il modello con la nostra funzione:

In [ ]:
model_1hl_A = create_nn_1h(n_feature, True)

Visualizziamo il modello della rete neurale appena creata:

In [ ]:
tf.keras.utils.plot_model(
    model_1hl_A,
    rankdir='LR',
    show_shapes=True,
    show_layer_names=True,
    show_dtype=True,
    show_layer_activations=True,
    show_trainable=True
)

Addestriamo la rete:

In [ ]:
training = model_1hl_A.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 32, epochs = 100, verbose=1, shuffle=True)

Valutiamo le prestazioni. Per farlo, possiamo ricorrere alla [history callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History) di Keras: è una callback che tiene traccia di tutto l'addestramento della rete e viene applicata automaticamente al momento dell'addestramento. Possiamo ottenere i parametri utilizzati e la storia dell'addestramento molto facilmente:

In [ ]:
training.params

In [ ]:
training.history

Come possiamo vedere, l'attributo **history** contiene i valori di cost function calcolati su training e test set in ogni epoca. E' molto semplice ottenere un grafico andando prima a convertire questo array in un dataframe e poi chiamando la funzione plot:

In [ ]:
loss = pd.DataFrame(training.history)
loss.plot()

Possiamo notare come il valore di MSE decresca all'aumentare dell'epoca di addestramento, come è desiderabile che sia. L'andamento ottenuto suggerisce che si potrebbero ottenere delle prestazioni migliori aumentando le epoche di addestramento: se la validation loss avesse avuto un andamento oscillatorio o, comunque, avesse presentato una risalita, questo avrebbe indicato un over-fitting.

Vediamo cosa accade con 400 epoche:

In [ ]:
training = model_1hl_A.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 32, epochs = 500, verbose=1, shuffle=True)

In [ ]:
loss = pd.DataFrame(training.history)
loss.plot()

Siamo arrivati ad una riduzione soddisfacente del costo, non avrebbe senso spingersi oltre (anzi, rischieremmo di andare in over-fitting). Calcoliamo $R^2$ e $\overline{R}^2$:

In [ ]:
from sklearn.metrics import r2_score

y_hat = model_1hl_A.predict(X_test, verbose=0)

r2 = r2_score(y_test, y_hat)
print(f'R2 = {r2}')

n_obs = X_train.shape[0]
r2_adj = 1-(1-r2)*(n_obs-1)/(n_obs-n_feature-1)
print(f'R2-adj = {r2_adj}')

Possiamo notare come, con una rete neurale relativamente semplice, abbiamo ottenuto circa le prestazioni ottimali di una regressione lineare o polinomiale multipla! Prima di valutare altre configurazioni per vedere se c'è un ulteriore margine di miglioramento, è opportuno ricorrere alla k-fold cross-validation; infatti, può facilmente accadere che questa stessa rete, testata più volte con holdout, porti a dei risultati estremamente diversi con fluttuazioni marcate.

Nell'applicare la k-fold cross-validation, settiamo **verbose=0** per i metodi fit() e predict() in modo da non avere stampe incontrollate ad ogni iterazione.

In [ ]:
from sklearn.model_selection import KFold

n_fold = 10
k_fold = KFold(n_splits=n_fold, random_state=0, shuffle=True)

# inizializziamo delle liste per le metriche
r2 = []
r2_adj = []

for train_index , test_index in k_fold.split(X): # iterazione su tutti i fold con unpacking di train/test index
    # split corrente
    X_train, X_test = X[train_index,:], X[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    # feature scaling sullo split corrente
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # addestramento e risultati
    training = model_1hl_A.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 32, epochs = 400, verbose=0, shuffle=True)
    # predizioni
    y_hat = model_1hl_A.predict(X_test, verbose=0)
    # metriche
    r2_temp = r2_score(y_test, y_hat)
    r2.append(r2_temp)
    n_obs = X_train.shape[0]
    r2_adj.append(1-(1-r2_temp)*(n_obs-1)/(n_obs-n_feature-1))
    
print(f"R2 = {np.mean(r2)}")
print(f"R2-adjusted = {np.mean(r2_adj)}")

Il valore di $\overline{R}^2$ è fondamentalmente quello ottenuto con holdout, per cui erano prestazioni piuttosto veritiere.

Abbiamo quindi ottenuto un risultato notevole considerando che c'è un singolo hidden layer...se fossimo già soddisfatti, potremmo addestrare il modello su tutto il dataset per ottenere la rete neurale finale per le predizioni:

In [ ]:
X_copy = sc.fit_transform(X)  # lo chiamiamo X_train solo per non sovrascrivere X, in realtà ci sono tutte le osservazioni
tf.keras.utils.set_random_seed(my_seed)
training = model_1hl_A.fit(X_copy, y, validation_split=0, batch_size = 32, epochs = 400, verbose=0, shuffle=True)
y_hat = model_1hl_A.predict(X_copy, verbose=0)
n_obs = X_copy.shape[0]
print(1-(1-r2_score(y, y_hat))*(n_obs-1)/(n_obs-n_feature-1))

Chiaramente il risultato è migliore: abbiamo ri-addestrato la rete su tutto il dataset senza dividere train e test.

Se siamo davvero soddisfatti, possiamo salvare il nostro modello in un file .keras tramite il metodo [save()](https://www.tensorflow.org/api_docs/python/tf/keras/models/save_model). Possiamo poi caricare il modello allenato con la funzione [load_model()](https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model):

In [ ]:
model_1hl_A.save('my_neural_network.keras')

In [ ]:
import keras
loaded_model = keras.models.load_model('my_neural_network.keras')

In [ ]:
X_copy = sc.fit_transform(X)
tf.keras.utils.set_random_seed(my_seed)
loaded_model.fit(X_copy, y, validation_split=0, batch_size = 32, epochs = 400, verbose=0, shuffle=True)
y_hat = loaded_model.predict(X_copy, verbose=0)
n_obs = X_copy.shape[0]
print(1-(1-r2_score(y, y_hat))*(n_obs-1)/(n_obs-n_feature-1))

Con gli accorgimenti sui seed da re-inizializzare, si ottengono assolutamente gli stessi risultati.

#### Soluzione B: $(M+L)*2/3$ nodi

Valutiamo la seconda opzione, con meno nodi nell'hidden layer. Non dimentichiamo di re-inizializzare i seed prima di creare la rete, in quanto la funzione da noi definita include la compilazione che è il momento in cui i pesi vengono inizializzati:

In [ ]:
tf.keras.utils.set_random_seed(my_seed)
model_1hl_B = create_nn_1h(int(2*(n_feature+1)/3), True)

In [ ]:
tf.keras.utils.plot_model(
    model_1hl_B,
    rankdir='LR',
    show_shapes=True,
    show_layer_names=True,
    show_dtype=True,
    show_layer_activations=True,
    show_trainable=True
)

In [ ]:
training = model_1hl_B.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 32, epochs = 500, verbose=1, shuffle=True)

In [ ]:
loss = pd.DataFrame(training.history)
loss.plot()

In [ ]:
y_hat = model_1hl_B.predict(X_test, verbose=0)

r2 = r2_score(y_test, y_hat)
print(f'R2 = {r2}')

n_obs = X_train.shape[0]
r2_adj = 1-(1-r2)*(n_obs-1)/(n_obs-n_feature-1)
print(f'R2-adj = {r2_adj}')

Le prestazioni sembrano leggermente migliori, ma per averne la certezza è opportuno verificare con una k-fold:

In [ ]:
n_fold = 10
k_fold = KFold(n_splits=n_fold, random_state=0, shuffle=True)

# inizializziamo delle liste per le metriche
r2 = []
r2_adj = []

for train_index , test_index in k_fold.split(X): # iterazione su tutti i fold con unpacking di train/test index
    # split corrente
    X_train , X_test = X[train_index,:],X[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
    # feature scaling sullo split corrente
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # addestramento e risultati
    training = model_1hl_B.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 32, epochs = 500, verbose=0, shuffle=True)
    # predizioni
    y_hat = model_1hl_B.predict(X_test, verbose=0)
    # metriche
    r2_temp = r2_score(y_test, y_hat)
    r2.append(r2_temp)
    n_obs = X_train.shape[0]
    r2_adj.append(1-(1-r2_temp)*(n_obs-1)/(n_obs-n_feature-1))
    
print(f"R2 = {np.mean(r2)}")
print(f"R2-adjusted = {np.mean(r2_adj)}")

Le prestazioni sono in linea con l'holdout e fondamentalmente le stesse del caso precedente. Non è una sorpresa, dato che abbiamo lo stesso design tranne un neurone in meno nell'hidden layer (tuttavia, con altri seed si potrebbero osservare delle differenze sostanziali che non devono allarmare: sono legate a diversi ottimi locali individuati, ma i trend dovrebbero rimanere i medesimi).

### 2 hidden layer

In questo caso, valutiamo due soluzioni: lo stesso numero di nodi per layer e un valore decrescente. Avendo 3 feature, consideriamo quindi un primo layer con 3 neuroni ed un secondo layer con 3 (caso A) o 2 (caso B) neuroni, dato che il caso di singolo nodo è fondamentalmente un output layer. Aggiorniamo la funzione che crea la rete per prendere due ingressi:

In [ ]:
def create_nn_2h(n_neuron_1: int, n_neuron_2: int, summary_flag: bool = True):
    """
    Questa funzione crea una rete neurale con due hidden layer. 
    :param n_neuron_1: numero di neuroni dell'hidden layer 1
    :param n_neuron_2: numero di neuroni dell'hidden layer 2
    :param summary_flag: flag booleano per indicare se mostrare il riassunto della rete oppure no
    :return: modello di rete neurale compilato con ottimizzatore 'adam' e loss function 'mse'
    """
    # layout della rete
    inputs = layers.Input(name = 'input_layer', shape=(n_feature,) )
    h1 = layers.Dense(name = 'hidden_layer_1', units = n_neuron_1, activation = 'relu')(inputs)
    h2 = layers.Dense(name = 'hidden_layer_2', units = n_neuron_2, activation = 'relu')(h1)
    outputs = layers.Dense(name = 'output_layer', units = 1, activation = 'linear')(h2)
    # creazione della rete
    model = models.Model(inputs = inputs, outputs = outputs, name = 'NN_1HL')
    # summary
    if summary_flag: model.summary(show_trainable=True)
    # compilazione
    model.compile(optimizer = 'adam', loss = 'mse')
    # ritorno 
    return model

#### Soluzione A: nodi costanti

In questo caso, entrambi gli hidden layer hanno n_feature nodi:

In [ ]:
tf.keras.utils.set_random_seed(my_seed)
model_2hl_A = create_nn_2h(int(n_feature), int(n_feature), False)

In [ ]:
tf.keras.utils.plot_model(
    model_2hl_A,
    rankdir='LR',
    show_shapes=True,
    show_layer_names=True,
    show_dtype=True,
    show_layer_activations=True,
    show_trainable=True
)

In [ ]:
training = model_2hl_A.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 32, epochs = 500, verbose=1, shuffle=True)

In [ ]:
loss = pd.DataFrame(training.history)
loss.plot()

In [ ]:
y_hat = model_2hl_A.predict(X_test, verbose=0)

r2 = r2_score(y_test, y_hat)
print(f'R2 = {r2}')

n_obs = X_train.shape[0]
r2_adj = 1-(1-r2)*(n_obs-1)/(n_obs-n_feature-1)
print(f'R2-adj = {r2_adj}')

Le prestazioni sembrano crollate...può accadere per come è calcolata questa metrica (si vedano le slide) quando fondamentalmente il predittore tiene in considerazione solo un'intercetta, fornendo sempre lo stesso risultato. Vediamo se aumentando le epoche di addestramento otteniamo un miglioramento.

In [ ]:
tf.keras.utils.set_random_seed(my_seed)
model_2hl_A = create_nn_2h(int(n_feature), int(n_feature), False)

training = model_2hl_A.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 32, epochs = 1500, verbose=0, shuffle=True)

y_hat = model_2hl_A.predict(X_test, verbose=0)

r2 = r2_score(y_test, y_hat)
print(f'R2 = {r2}')

n_obs = X_train.shape[0]
r2_adj = 1-(1-r2)*(n_obs-1)/(n_obs-n_feature-1)
print(f'R2-adj = {r2_adj}')

Ci rimane solo da verificare con un k-fold se è davvero questa la situazione:

In [ ]:
n_fold = 10
k_fold = KFold(n_splits=n_fold, random_state=0, shuffle=True)

# inizializziamo delle liste per le metriche
r2 = []
r2_adj = []

for train_index , test_index in k_fold.split(X): # iterazione su tutti i fold con unpacking di train/test index
    # split corrente
    X_train , X_test = X[train_index,:],X[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
    # feature scaling sullo split corrente
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # addestramento e risultati
    training = model_2hl_A.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 32, epochs = 900, verbose=0, shuffle=True)
    # predizioni
    y_hat = model_2hl_A.predict(X_test, verbose=0)
    # metriche
    r2_temp = r2_score(y_test, y_hat)
    r2.append(r2_temp)
    n_obs = X_train.shape[0]
    r2_adj.append(1-(1-r2_temp)*(n_obs-1)/(n_obs-n_feature-1))
    
print(f"R2 = {np.mean(r2)}")
print(f"R2-adjusted = {np.mean(r2_adj)}")

Le prestazioni sono sicuramente migliorate rispetto ad un approccio holdout, ma rimaniamo pur sempre con un valore di $\overline{R}^2$ negativo.

#### Soluzione B: nodi decrescenti

Vediamo se riducendo di una unità il numero di nodi del secondo hidden layer le prestazioni cambiano:

In [ ]:
tf.keras.utils.set_random_seed(my_seed)
model_2hl_B = create_nn_2h(int(n_feature), int(n_feature-1), False)

In [ ]:
tf.keras.utils.plot_model(
    model_2hl_B,
    rankdir='LR',
    show_shapes=True,
    show_layer_names=True,
    show_dtype=True,
    show_layer_activations=True,
    show_trainable=True
)

In [ ]:
training = model_2hl_B.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 32, epochs = 500, verbose=1, shuffle=True)

In [ ]:
loss = pd.DataFrame(training.history)
loss.plot()

In [ ]:
y_hat = model_2hl_B.predict(X_test)

r2 = r2_score(y_test, y_hat)
print(f'R2 = {r2}')

n_obs = X_train.shape[0]
r2_adj = 1-(1-r2)*(n_obs-1)/(n_obs-n_feature-1)
print(f'R2-adj = {r2_adj}')

Sembra di essere nelle stesse condizioni di prima:

In [ ]:
training = model_2hl_B.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 32, epochs = 900, verbose=1, shuffle=True)

In [ ]:
loss = pd.DataFrame(training.history)
loss.plot()

In [ ]:
y_hat = model_2hl_B.predict(X_test)

r2 = r2_score(y_test, y_hat)
print(f'R2 = {r2}')

n_obs = X_train.shape[0]
r2_adj = 1-(1-r2)*(n_obs-1)/(n_obs-n_feature-1)
print(f'R2-adj = {r2_adj}')

Non sembrerebbe una rete funzionante...valutiamo anche con k-fold per sicurezza:

In [ ]:
n_fold = 10
k_fold = KFold(n_splits=n_fold, random_state=0, shuffle=True)

# inizializziamo delle liste per le metriche
r2 = []
r2_adj = []

for train_index , test_index in k_fold.split(X): # iterazione su tutti i fold con unpacking di train/test index
    # split corrente
    X_train , X_test = X[train_index,:],X[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
    # feature scaling sullo split corrente
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # addestramento e risultati
    training = model_2hl_B.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 32, epochs = 900, verbose=0, shuffle=True)
    # predizioni
    y_hat = model_2hl_B.predict(X_test, verbose=0)
    # metriche
    r2_temp = r2_score(y_test, y_hat)
    r2.append(r2_temp)
    n_obs = X_train.shape[0]
    r2_adj.append(1-(1-r2_temp)*(n_obs-1)/(n_obs-n_feature-1))
    
print(f"R2 = {np.mean(r2)}")
print(f"R2-adjusted = {np.mean(r2_adj)}")

Con una k-fold cross-validation, le prestazioni sono in linea con le precedenti. Si potrebbe andare avanti così per molto tempo...o ricorrere all'ottimizzazione degli iper-parametri!

# Ottimizzazione degli iper-parametri

Ottimizzare gli iper-parametri in una rete neurale è simile al caso con Machine Learning. Tuttavia, vi sono un paio di aspetti a cui prestare attenzione, come vedremo. Inoltre, l'addestramento di una rete neurale è più oneroso come tempi di calcolo; per questo, valuteremo alcuni casi in modo separato, invece di seguire una grid search estensiva che richiede molto tempo ed è l'approccio da seguire.

## Architettura della rete

Prima di tutto, creiamo una funzione che ci permette di creare una rete neurale con un numero di hidden layer e di nodi variabile; per ogni hidden layer, inseriamo lo stesso numero di neuroni per pura comodità. Per riprodurre i risultati, inseriamo la re-inizializzazione dei seed nella funzione dato che avrà un impatto ogni volta che viene creato un modello.

In [ ]:
def create_nn(n_hidden: int, n_neuron: int):
    """
    Questa funzione crea una rete neurale con un numero di hidden layer e di nodi variabile.
    :param n_hidden: numero di hidden layer 
    :param n_neuron: numero di neuroni per hidden layer
    :return: modello di rete neurale compilato con ottimizzatore 'adam' e loss function 'mse'
    """
    tf.keras.utils.set_random_seed(my_seed)
    # layout della rete: input layer
    inputs = layers.Input(name = 'input_layer', shape=(n_feature,) )
    # layout della rete: hidden layer
    hidden = [layers.Dense(name='hidden_layer_1', units=n_feature, activation='relu')(inputs)]
    for n in range(2, n_hidden+1):
        hidden.append(layers.Dense(name = f'hidden_layer_{n}', units = n_neuron, activation = 'relu')(hidden[n-2]))
    # layout della rete: output layer
    outputs = layers.Dense(name = 'output_layer', units = 1, activation = 'linear')(hidden[-1])
    # creazione della rete
    model = models.Model(inputs = inputs, outputs = outputs, name = 'NN_model')
    # compilazione
    model.compile(optimizer = 'adam', loss = 'mse')
    # ritorno 
    return model

Una volta definita la funzione, la passiamo in ingresso ad un wrapper (decoratore) di Keras fornito dal package [scikeras](https://adriangb.com/scikeras/stable/): [KerasRegressor](https://adriangb.com/scikeras/stable/generated/scikeras.wrappers.KerasRegressor.html). Questo ci permetterà di usare le API di Keras per l'ottimizzazione degli iper-parametri. Bisogna prestare attenzione ad un aspetto: i parametri da ottimizzare, anche quando sono argomenti di ingresso della funzione, devono essere inseriti con un qualche valore anche nella creazione dell'oggetto KerasRegressor. Senza questa dichiarazione, l'addestramento della grid search fallirà.

In [ ]:
from scikeras.wrappers import KerasRegressor

model = KerasRegressor(model=create_nn, batch_size=32, epochs=400, verbose=0, n_hidden=1, n_neuron=2)

A questo punto, definiamo i parametri della ricerca:

In [ ]:
param_grid = {
    'n_hidden': [1, 2, 3, 4],
    'n_neuron': [2, 3]
}

Creiamo un oggetto GridSearchCV come visto già per il Machine Learning:

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, error_score=0.0, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

Nel lanciare la grid search, utilizziamo tutto il dataset in quanto lo split viene fatto internamente con una k-fold.

In [ ]:
sc = StandardScaler()
X_copy = sc.fit_transform(X)
grid_result = grid_search.fit(X_copy, y, verbose=0)

La configurazione migliore è salvata nell'attributo **best_params_** come un dizionario:

In [ ]:
grid_result.best_params_

Il risultato migliore è invece contenuto in **best_score_**: attenzione perché la grid search massimizza la funzione, invece di minimizzarla (infatti la metrica è negativa anche nel nome con cui viene passata). Per avere il valore di MSE effettivo, si deve prendere il negativo del risultato:

In [ ]:
-grid_result.best_score_

Infine, **best_estimator_** contiene lo stimatore ottimale, che possiamo utilizzare per fare predizioni su osservazioni mai viste (o per ottenere il valore di $\overline{R}^2$ del miglior modello):

In [ ]:
best_model = grid_result.best_estimator_

In [ ]:
y_hat = grid_result.best_estimator_.predict(X_copy, verbose=0)
n_obs = X_copy.shape[0]
print(1-(1-r2_score(y, y_hat))*(n_obs-1)/(n_obs-n_feature-1))

## Addestramento della rete

E' anche possibile ottimizzare gli iper-parametri relativi all'apprendimento della rete, come ad esempio il numero di epoche, di batch, l'ottimizzatore, la sua learning rate, etc. Tecnicamente, andrebbe fatto tutto in un'unica ricerca, dato che la definizione di un parametro influisce sulle prestazioni degli altri...ma per comodità, creiamo una rete con la configurazione ottimale ottenuta prima e vediamo come ottimizzare il numero di epoche e il batch size; la rete ottimale può essere definita indicando i suoi parametri nel costruttore KerasRegressor.

In [ ]:
param_grid = {
    'batch_size': [32, 64],
    'epochs': [400, 500],
}

In [ ]:
model = KerasRegressor(model=create_nn, verbose=0, n_hidden=4, n_neuron=3)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, error_score=0.0, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

In [ ]:
sc = StandardScaler()
X_copy = sc.fit_transform(X)
grid_result = grid_search.fit(X_copy, y, verbose=0)

In [ ]:
grid_result.best_params_

In [ ]:
-grid_result.best_score_

Per quanto riguarda l'ottimizzatore, questo compare nella compilazione del modello...dobbiamo quindi modificare leggermente la nostra funzione per prendere questo come parametro in ingresso. Inoltre bisogna prestare attenzione ai nomi dei keyword argument passati tramite il dizionario dei parametri:
- la learning rate invece deve essere indicata dalla chiave **optimizer__learning_rate**;
- l'ottimizzatore è indicato come **model__optimizer**.

In [ ]:
def create_nn(n_hidden: int, n_neuron: int, optimizer: str):
    """
    Questa funzione crea una rete neurale con un numero di hidden layer e di nodi variabile.
    :param n_hidden: numero di hidden layer 
    :param n_neuron: numero di neuroni per hidden layer
    :param optimizer: ottimizzatore per l'addestramento
    :return: modello di rete neurale compilato con ottimizzatore 'adam' e loss function 'mse'
    """
    tf.keras.utils.set_random_seed(my_seed)
    # layout della rete: input layer
    inputs = layers.Input(name = 'input_layer', shape=(n_feature,) )
    # layout della rete: hidden layer
    hidden = [layers.Dense(name='hidden_layer_1', units=n_feature, activation='relu')(inputs)]
    for n in range(2, n_hidden+1):
        hidden.append(layers.Dense(name = f'hidden_layer_{n}', units = n_neuron, activation = 'relu')(hidden[n-2]))
    # layout della rete: output layer
    outputs = layers.Dense(name = 'output_layer', units = 1, activation = 'linear')(hidden[-1])
    # creazione della rete
    model = models.Model(inputs = inputs, outputs = outputs, name = 'NN_model')
    # compilazione
    model.compile(optimizer = optimizer, loss = 'mse')
    # ritorno 
    return model

In [ ]:
param_grid = {
    'model__optimizer': ['adam', 'SGD'],
    'optimizer__learning_rate': [0.001, 0.01],
}

In [ ]:
model = KerasRegressor(model=create_nn, verbose=0, n_hidden=4, n_neuron=3, optimizer='adam', epochs=500, batch_size=32)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, error_score=0.0, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

In [ ]:
sc = StandardScaler()
X_copy = sc.fit_transform(X)
grid_result = grid_search.fit(X_copy, y, verbose=0)

In [ ]:
grid_result.best_params_

In [ ]:
-grid_result.best_score_

Abbiamo ottenuto la configurazione precedente come ottimale, per cui i risultati sono gli stessi.